In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import threading
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import pyttsx3 # Text-to-speech library
import os
import datetime
import pandas as pd
from transformers import pipeline
from googletrans import Translator

# Load trained model
def load_trained_model(model_path='plant_disease_model_fast.h5'):
    try:
        model = load_model(model_path, compile=False)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print(f"Model load failed: {e}")
        return None

# Predict disease from image
def predict_disease(model, img_path, image_size=(128, 128), class_names=None):
    try:
        img = image.load_img(img_path, target_size=image_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        predictions = model.predict(img_array)
        predicted_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_index] if class_names else str(predicted_index)
        confidence = predictions[0][predicted_index]
        return predicted_class, confidence
    except Exception as e:
        return "Error", 0.0

# Thread-safe voice output
def speak_text(text, lang='en'):
    def _speak():
        engine = pyttsx3.init()
        engine.setProperty('rate', 150)
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak, daemon=True).start()

# Simulated voice command input (stub)
def listen_command():
    return input("Enter voice command (simulated): ")

# Log session data
def append_log(farmer_name, location, crop, problem, disease_pred, confidence, advice):
    log_file = "farmer_app_log.csv"
    log_data = {
        "Date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
        "Farmer Name": farmer_name,
        "Location": location,
        "Crop": crop,
        "Problem": problem,
        "Prediction": disease_pred,
        "Confidence": f"{confidence:.2f}",
        "Advice": advice
    }
    file_exists = os.path.isfile(log_file)
    df = pd.DataFrame([log_data])
    df.to_csv(log_file, mode='a', header=not file_exists, index=False)

class FarmerAssistantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Offline Farmer Assistant")
        self.root.geometry("750x700")

        self.class_names = [
            "Healthy", "Early Blight", "Late Blight",
            "Powdery Mildew", "Leaf Spot", "Rust",
            "Bacterial Spot", "Septoria Leaf Spot", "Yellow Leaf Curl Virus"
        ]

        self.translator = Translator()
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        self.labels = ["nutrient deficiency", "pest attack", "fungal infection", "watering issue", "unknown issue"]

        self.languages = {"English": "en", "Hindi": "hi", "Spanish": "es", "French": "fr", "Arabic": "ar"}

        self.model = load_trained_model()
        if not self.model:
            messagebox.showwarning("Model Warning", "Model not loaded. Prediction won't work.")

        self.create_widgets()

    def create_widgets(self):
        frame_lang = ttk.LabelFrame(self.root, text="🌐 Language Selection")
        frame_lang.pack(fill='x', padx=10, pady=5)

        ttk.Label(frame_lang, text="Language:").grid(row=0, column=0, padx=5, pady=5)
        self.lang_var = tk.StringVar()
        self.lang_dropdown = ttk.Combobox(frame_lang, textvariable=self.lang_var, values=list(self.languages.keys()), state="readonly")
        self.lang_dropdown.grid(row=0, column=1, padx=5, pady=5)
        self.lang_dropdown.set("English")

        frame_info = ttk.LabelFrame(self.root, text="👤 Farmer Details")
        frame_info.pack(fill='x', padx=10, pady=5)

        ttk.Label(frame_info, text="Name:").grid(row=0, column=0, sticky='w', padx=5, pady=5)
        self.name_entry = ttk.Entry(frame_info)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(frame_info, text="Location:").grid(row=0, column=2, sticky='w', padx=5, pady=5)
        self.location_entry = ttk.Entry(frame_info)
        self.location_entry.grid(row=0, column=3, padx=5, pady=5)

        frame_selection = ttk.LabelFrame(self.root, text="🌾 Crop Selection")
        frame_selection.pack(fill='x', padx=10, pady=5)

        ttk.Label(frame_selection, text="Crop:").grid(row=0, column=0, padx=5, pady=5)
        crops = ["Tomato", "Potato", "Wheat", "Rice", "Brinjal", "Cotton", "Maize"]
        self.crop_var = tk.StringVar()
        self.crop_dropdown = ttk.Combobox(frame_selection, textvariable=self.crop_var, values=crops, state="readonly")
        self.crop_dropdown.grid(row=0, column=1, padx=5, pady=5)
        self.crop_dropdown.set(crops[0])

        self.upload_btn = ttk.Button(self.root, text="📸 Upload Crop Image for Diagnosis", command=self.upload_image)
        self.upload_btn.pack(pady=10)

        self.img_label = ttk.Label(self.root)
        self.img_label.pack(pady=5)

        self.result_text = tk.Text(self.root, height=12, width=85)
        self.result_text.pack(pady=10)

        self.voice_btn = ttk.Button(self.root, text="🎤 Use Voice Command", command=self.handle_voice_command)
        self.voice_btn.pack(pady=5)

    def upload_image(self):
        file_path = filedialog.askopenfilename(
            title="Select Crop Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if file_path:
            self.display_image(file_path)
            threading.Thread(target=self.run_prediction, args=(file_path,), daemon=True).start()

    def display_image(self, img_path):
        img = Image.open(img_path).resize((250, 250))
        img_tk = ImageTk.PhotoImage(img)
        self.img_label.configure(image=img_tk)
        self.img_label.image = img_tk

    def run_prediction(self, img_path):
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, "Predicting disease, please wait...\n")

        disease, confidence = predict_disease(self.model, img_path, class_names=self.class_names)
        advice_en = self.get_advice(disease)

        # NLP Classification of Problem
        nlp_result = self.classifier(disease, self.labels)
        problem_label = nlp_result['labels'][0]

        target_lang = self.languages[self.lang_var.get()]

        try:
            disease_translated = self.translator.translate(disease, dest=target_lang).text
            advice_translated = self.translator.translate(advice_en, dest=target_lang).text
            problem_translated = self.translator.translate(problem_label, dest=target_lang).text
        except:
            disease_translated = disease
            advice_translated = advice_en
            problem_translated = problem_label

        result_str = (f"Disease Prediction: {disease_translated}\n"
                      f"Confidence: {confidence*100:.2f}%\n"
                      f"Problem/Issue Detected: {problem_translated}\n"
                      f"Advice: {advice_translated}")

        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result_str)

        speak_text(f"Detected {disease_translated} with {int(confidence*100)} percent confidence.", lang=target_lang)
        speak_text(problem_translated, lang=target_lang)
        speak_text(advice_translated, lang=target_lang)

        append_log(
            farmer_name=self.name_entry.get(),
            location=self.location_entry.get(),
            crop=self.crop_var.get(),
            problem=problem_label,
            disease_pred=disease_translated,
            confidence=confidence,
            advice=advice_translated
        )

    def get_advice(self, disease):
        advice_dict = {
            "Early Blight": "Apply fungicides and remove infected leaves.",
            "Late Blight": "Use certified seeds and avoid overhead watering.",
            "Powdery Mildew": "Spray neem oil and improve air circulation.",
            "Leaf Spot": "Use appropriate fungicides and keep foliage dry.",
            "Rust": "Apply sulfur-based fungicide. Avoid wet conditions.",
            "Healthy": "Your crop is healthy. No action needed.",
            "Yellow Leaf Curl Virus": "Control whiteflies and remove infected plants.",
            "Bacterial Spot": "Use copper sprays and resistant varieties.",
            "Septoria Leaf Spot": "Remove affected leaves and spray fungicide.",
            "Error": "Could not predict disease. Try again."
        }
        return advice_dict.get(disease, "Consult local agricultural expert.")

    def handle_voice_command(self):
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, "Listening for voice command...\n")
        command = listen_command()
        if command:
            self.result_text.insert(tk.END, f"Voice command: {command}\n")
            if "diagnose" in command.lower():
                self.result_text.insert(tk.END, "Please upload an image using the button above.\n")
            else:
                self.result_text.insert(tk.END, "Command not recognized.\n")
        else:
            self.result_text.insert(tk.END, "No voice input detected.\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = FarmerAssistantApp(root)
    root.mainloop()

Device set to use cpu


Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
# ============================📦 LIBRARY IMPORTS =============================

# GUI development
import tkinter as tk  # Core tkinter module for windowing
from tkinter import ttk, filedialog, messagebox  # Styled widgets and dialogs

# Image handling
from PIL import Image, ImageTk  # For image opening and rendering in GUI

# Threading for background tasks (like speech)
import threading

# Array operations
import numpy as np

# Deep learning model loading and preprocessing
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Text-to-speech engine
import pyttsx3  # Converts text to speech offline

# File handling and logging
import os
import datetime
import pandas as pd

# NLP pipeline for problem classification
from transformers import pipeline  # HuggingFace transformer pipelines

# Language translation (Note: consider replacing with deep_translator if issues arise)
from googletrans import Translator

# ============================🧠 FUNCTION DEFINITIONS =============================

# Load a pre-trained disease classification model
def load_trained_model(model_path='plant_disease_model_fast.h5'):
    try:
        model = load_model(model_path, compile=False)
        print("Model loaded successfully.")
        return model
    except Exception as e:
        print(f"Model load failed: {e}")
        return None

# Predict disease from input image using trained model
def predict_disease(model, img_path, image_size=(128, 128), class_names=None):
    try:
        img = image.load_img(img_path, target_size=image_size)
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0) / 255.0
        predictions = model.predict(img_array)
        predicted_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_index] if class_names else str(predicted_index)
        confidence = predictions[0][predicted_index]
        return predicted_class, confidence
    except Exception as e:
        return "Error", 0.0

# Speak output using TTS (non-blocking)
def speak_text(text, lang='en'):
    def _speak():
        engine = pyttsx3.init()
        engine.setProperty('rate', 150)
        engine.say(text)
        engine.runAndWait()
    threading.Thread(target=_speak, daemon=True).start()

# Placeholder function for voice command input (manual)
def listen_command():
    return input("Enter voice command (simulated): ")

# Save session data to CSV log file
def append_log(farmer_name, location, crop, problem, disease_pred, confidence, advice):
    log_file = "farmer_app_log.csv"
    log_data = {
        "Date": datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
        "Farmer Name": farmer_name,
        "Location": location,
        "Crop": crop,
        "Problem": problem,
        "Prediction": disease_pred,
        "Confidence": f"{confidence:.2f}",
        "Advice": advice
    }
    file_exists = os.path.isfile(log_file)
    df = pd.DataFrame([log_data])
    df.to_csv(log_file, mode='a', header=not file_exists, index=False)

# ============================🌾 FARMER ASSISTANT GUI CLASS =============================

class FarmerAssistantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Offline Farmer Assistant")
        self.root.geometry("750x700")

        # Custom class labels matching your trained model
        self.class_names = [
            "Healthy", "Early Blight", "Late Blight",
            "Powdery Mildew", "Leaf Spot", "Rust",
            "Bacterial Spot", "Septoria Leaf Spot", "Yellow Leaf Curl Virus"
        ]

        # Translator and NLP classifier setup
        self.translator = Translator()
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
        self.labels = ["nutrient deficiency", "pest attack", "fungal infection", "watering issue", "unknown issue"]

        # Language mapping for GUI
        self.languages = {"English": "en", "Hindi": "hi", "Spanish": "es", "French": "fr", "Arabic": "ar"}

        # Load model on startup
        self.model = load_trained_model()
        if not self.model:
            messagebox.showwarning("Model Warning", "Model not loaded. Prediction won't work.")

        # Build GUI elements
        self.create_widgets()

    # GUI Layout setup
    def create_widgets(self):
        # Language selection dropdown
        frame_lang = ttk.LabelFrame(self.root, text="🌐 Language Selection")
        frame_lang.pack(fill='x', padx=10, pady=5)

        ttk.Label(frame_lang, text="Language:").grid(row=0, column=0, padx=5, pady=5)
        self.lang_var = tk.StringVar()
        self.lang_dropdown = ttk.Combobox(frame_lang, textvariable=self.lang_var, values=list(self.languages.keys()), state="readonly")
        self.lang_dropdown.grid(row=0, column=1, padx=5, pady=5)
        self.lang_dropdown.set("English")

        # Farmer info
        frame_info = ttk.LabelFrame(self.root, text="👤 Farmer Details")
        frame_info.pack(fill='x', padx=10, pady=5)

        ttk.Label(frame_info, text="Name:").grid(row=0, column=0, sticky='w', padx=5, pady=5)
        self.name_entry = ttk.Entry(frame_info)
        self.name_entry.grid(row=0, column=1, padx=5, pady=5)

        ttk.Label(frame_info, text="Location:").grid(row=0, column=2, sticky='w', padx=5, pady=5)
        self.location_entry = ttk.Entry(frame_info)
        self.location_entry.grid(row=0, column=3, padx=5, pady=5)

        # Crop selection
        frame_selection = ttk.LabelFrame(self.root, text="🌾 Crop Selection")
        frame_selection.pack(fill='x', padx=10, pady=5)

        ttk.Label(frame_selection, text="Crop:").grid(row=0, column=0, padx=5, pady=5)
        crops = ["Tomato", "Potato", "Wheat", "Rice", "Brinjal", "Cotton", "Maize"]
        self.crop_var = tk.StringVar()
        self.crop_dropdown = ttk.Combobox(frame_selection, textvariable=self.crop_var, values=crops, state="readonly")
        self.crop_dropdown.grid(row=0, column=1, padx=5, pady=5)
        self.crop_dropdown.set(crops[0])

        # Upload image button
        self.upload_btn = ttk.Button(self.root, text="📸 Upload Crop Image for Diagnosis", command=self.upload_image)
        self.upload_btn.pack(pady=10)

        self.img_label = ttk.Label(self.root)
        self.img_label.pack(pady=5)

        # Result display box
        self.result_text = tk.Text(self.root, height=12, width=85)
        self.result_text.pack(pady=10)

        # Voice interaction
        self.voice_btn = ttk.Button(self.root, text="🎤 Use Voice Command", command=self.handle_voice_command)
        self.voice_btn.pack(pady=5)

    # Upload image and trigger diagnosis
    def upload_image(self):
        file_path = filedialog.askopenfilename(
            title="Select Crop Image",
            filetypes=[("Image files", "*.jpg *.jpeg *.png")]
        )
        if file_path:
            self.display_image(file_path)
            threading.Thread(target=self.run_prediction, args=(file_path,), daemon=True).start()

    # Show image in the GUI
    def display_image(self, img_path):
        img = Image.open(img_path).resize((250, 250))
        img_tk = ImageTk.PhotoImage(img)
        self.img_label.configure(image=img_tk)
        self.img_label.image = img_tk

    # Perform prediction and show results
    def run_prediction(self, img_path):
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, "Predicting disease, please wait...\n")

        disease, confidence = predict_disease(self.model, img_path, class_names=self.class_names)
        advice_en = self.get_advice(disease)

        # NLP Classification for broader issue detection
        nlp_result = self.classifier(disease, self.labels)
        problem_label = nlp_result['labels'][0]

        target_lang = self.languages[self.lang_var.get()]

        try:
            disease_translated = self.translator.translate(disease, dest=target_lang).text
            advice_translated = self.translator.translate(advice_en, dest=target_lang).text
            problem_translated = self.translator.translate(problem_label, dest=target_lang).text
        except:
            disease_translated = disease
            advice_translated = advice_en
            problem_translated = problem_label

        result_str = (f"Disease Prediction: {disease_translated}\n"
                      f"Confidence: {confidence*100:.2f}%\n"
                      f"Problem/Issue Detected: {problem_translated}\n"
                      f"Advice: {advice_translated}")

        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, result_str)

        # Voice output
        speak_text(f"Detected {disease_translated} with {int(confidence*100)} percent confidence.", lang=target_lang)
        speak_text(problem_translated, lang=target_lang)
        speak_text(advice_translated, lang=target_lang)

        # Save log
        append_log(
            farmer_name=self.name_entry.get(),
            location=self.location_entry.get(),
            crop=self.crop_var.get(),
            problem=problem_label,
            disease_pred=disease_translated,
            confidence=confidence,
            advice=advice_translated
        )

    # Basic rules for disease treatment
    def get_advice(self, disease):
        advice_dict = {
            "Early Blight": "Apply fungicides and remove infected leaves.",
            "Late Blight": "Use certified seeds and avoid overhead watering.",
            "Powdery Mildew": "Spray neem oil and improve air circulation.",
            "Leaf Spot": "Use appropriate fungicides and keep foliage dry.",
            "Rust": "Apply sulfur-based fungicide. Avoid wet conditions.",
            "Healthy": "Your crop is healthy. No action needed.",
            "Yellow Leaf Curl Virus": "Control whiteflies and remove infected plants.",
            "Bacterial Spot": "Use copper sprays and resistant varieties.",
            "Septoria Leaf Spot": "Remove affected leaves and spray fungicide.",
            "Error": "Could not predict disease. Try again."
        }
        return advice_dict.get(disease, "Consult local agricultural expert.")

    # Handle simulated voice commands (stub)
    def handle_voice_command(self):
        self.result_text.delete(1.0, tk.END)
        self.result_text.insert(tk.END, "Listening for voice command...\n")
        command = listen_command()
        if command:
            self.result_text.insert(tk.END, f"Voice command: {command}\n")
            if "diagnose" in command.lower():
                self.result_text.insert(tk.END, "Please upload an image using the button above.\n")
            else:
                self.result_text.insert(tk.END, "Command not recognized.\n")
        else:
            self.result_text.insert(tk.END, "No voice input detected.\n")

# ============================🏁 LAUNCH THE APP =============================
if __name__ == "__main__":
    root = tk.Tk()
    app = FarmerAssistantApp(root)
    root.mainloop()
